# Terry Stop Classification Model

A Terry Stop, also known as a "Stop, Question, and Frisk" or a "Stop and Frisk" is based on the Supreme Court case *Terry V. Ohio*. The court in Terry held that a stop and frisk must comply with the Fourth Amendment, meaning that the stop-and-frisk cannot be unreasonable.

![Stop and frisk](./data/th.jpg)

A reasonable stop-and-frisk is one in which a reasonably prudent office is endangered. He/she may make a reasonable search for weapons of the person believed by him to be armed and dangerous.

When an officer has a *reasonable suspicion* that an individual is armed, engaged in, or about to be engaged in criminal conduct, the office may briefly stop and detain an individual for a *pat-down*

In a traffic stop setting, the Terry condition of a lawful investigatory stop is met whenever it is lawful for the police to detain an automobile and its occupants pending inquiry into a vehicular violation. The police do not need to believe that any occupant of the vehicle is involved in criminal activity. The reasonable suspicion of criminal activities discovered during the traffic stop may give rise to a Terry stop.

## Exlporatory Data Analysis
We'll use a number of Datasets, Seattle PD and NYPD. We'll process the data and create a classification that will help predict whether an arrest was made after a Terry Stop,

### Import the required Datasets and libraries

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#import the seattle PD data and print the first 5 rows
seattle_data = pd.read_csv("./data/Terry_Stops_20241018.csv")
nypd_data = pd.read_excel("./data/NYPD_terry_stops.xlsx")

seattle_data.head()

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,26 - 35,-1,20150000005870,101060,Field Contact,NaN,4980,1962,F,White,...,15:12:00.0000000,-,-,-,WEST PCT 2ND W - QUEEN - PLATOON 1,N,N,-,-,-
1,26 - 35,-1,20150000089918,28823,Arrest,NaN,5143,1957,M,Black or African American,...,19:13:00.0000000,ROBBERY - CRITICAL (INCLUDES STRONG ARM),--ROBBERY - STRONG ARM,911,NaN,N,Y,East,C,C1
2,-,-1,20180000078041,463764,Arrest,NaN,6921,1978,M,Asian,...,04:27:00.0000000,TRESPASS,--WARRANT SERVICES - MISDEMEANOR,911,SOUTH PCT 3RD W - ROBERT - PLATOON 2,N,Y,South,R,R2
3,56 and Above,12770971382,20200000175799,13267969395,Arrest,-,7266,1976,M,White,...,11:17:19.0000000,ASLT - CRITICAL (NO SHOOTINGS),"--ASSAULTS, OTHER",911,WEST PCT 1ST W - KING - PLATOON 1,Y,N,West,K,K1
4,26 - 35,-1,20160000108160,141119,Offense Report,NaN,7288,1984,M,White,...,17:35:00.0000000,"NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)",--SUSPICIOUS CIRCUM. - SUSPICIOUS VEHICLE,911,EAST PCT 2ND W - E/G RELIEF,N,N,East,E,E2


In [6]:
seattle_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61555 entries, 0 to 61554
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         61555 non-null  object
 1   Subject ID                61555 non-null  int64 
 2   GO / SC Num               61555 non-null  int64 
 3   Terry Stop ID             61555 non-null  int64 
 4   Stop Resolution           61555 non-null  object
 5   Weapon Type               28990 non-null  object
 6   Officer ID                61555 non-null  object
 7   Officer YOB               61555 non-null  int64 
 8   Officer Gender            61555 non-null  object
 9   Officer Race              61555 non-null  object
 10  Subject Perceived Race    61555 non-null  object
 11  Subject Perceived Gender  61555 non-null  object
 12  Reported Date             61555 non-null  object
 13  Reported Time             61555 non-null  object
 14  Initial Call Type     

In [7]:
seattle_data.describe()

,Subject ID,GO / SC Num,Terry Stop ID,Officer YOB
count,6.155500e+04,6.155500e+04,6.155500e+04,61555.000000
mean,7.514672e+09,2.018711e+13,1.258517e+10,1984.135976
std,1.307486e+10,8.549706e+10,1.800415e+10,9.467342
min,-8.000000e+00,-1.000000e+00,2.802000e+04,1900.000000
25%,-1.000000e+00,2.017000e+13,2.413675e+05,1979.000000
50%,-1.000000e+00,2.018000e+13,5.132840e+05,1986.000000
75%,7.755940e+09,2.021000e+13,2.040625e+10,1991.000000
max,6.008468e+10,2.024000e+13,6.008639e+10,2002.000000


In [15]:
seattle_data.isna().sum()

Subject Age Group               0
Subject ID                      0
GO / SC Num                     0
Terry Stop ID                   0
Stop Resolution                 0
Weapon Type                 32565
Officer ID                      0
Officer YOB                     0
Officer Gender                  0
Officer Race                    0
Subject Perceived Race          0
Subject Perceived Gender        0
Reported Date                   0
Reported Time                   0
Initial Call Type               0
Final Call Type                 0
Call Type                       0
Officer Squad                 561
Arrest Flag                     0
Frisk Flag                      0
Precinct                        0
Sector                          0
Beat                            0
dtype: int64

In [12]:
nypd_data.head()

,STOP_ID,STOP_FRISK_DATE,STOP_FRISK_TIME,YEAR2,MONTH2,DAY2,STOP_WAS_INITIATED,RECORD_STATUS_CODE,ISSUING_OFFICER_RANK,ISSUING_OFFICER_COMMAND_CODE,...,SUSPECT_OTHER_DESCRIPTION,STOP_LOCATION_PRECINCT,STOP_LOCATION_SECTOR_CODE,STOP_LOCATION_APARTMENT,STOP_LOCATION_FULL_ADDRESS,STOP_LOCATION_STREET_NAME,STOP_LOCATION_X,STOP_LOCATION_Y,STOP_LOCATION_PATROL_BORO_NAME,STOP_LOCATION_BORO_NAME
0,1,2023-01-01,00:44:00,2023,January,Sunday,Based on Radio Run,APP,POM,804,...,BLUE HOODIE BLUE JEANS,5,A,3B,46 MADISON ST,MADISON ST,984781,198509,PBMS,MANHATTAN
1,2,2023-01-01,00:49:00,2023,January,Sunday,Based on Self Initiated,APP,POM,69,...,UNK,69,B,(null),9910 SEAVIEW AVE,SEAVIEW AVE,1015557,170379,PBBS,BROOKLYN
2,3,2023-01-01,05:31:00,2023,January,Sunday,Based on Radio Run,APP,POM,83,...,NaN,83,D,(null),TROUTMAN ST && ST NICHOLAS AVE,TROUTMAN ST,1005940,197103,PBBN,BROOKLYN
3,4,2023-01-01,04:59:00,2023,January,Sunday,Based on Self Initiated,APP,POM,113,...,(null),113,B,(null),FARMERS BLVD && 135 AVE,FARMERS BLVD,1049830,185439,PBQS,QUEENS
4,5,2023-01-01,05:21:00,2023,January,Sunday,Based on Self Initiated,APP,POF,46,...,(null),46,B,(null),2463 WEBSTER AVE,WEBSTER AVE,1013821,252710,PBBX,BRONX


In [13]:
nypd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16971 entries, 0 to 16970
Data columns (total 82 columns):
 #   Column                                                        Non-Null Count  Dtype 
---  ------                                                        --------------  ----- 
 0   STOP_ID                                                       16971 non-null  int64 
 1   STOP_FRISK_DATE                                               16971 non-null  object
 2   STOP_FRISK_TIME                                               16971 non-null  object
 3   YEAR2                                                         16971 non-null  int64 
 4   MONTH2                                                        16971 non-null  object
 5   DAY2                                                          16971 non-null  object
 6   STOP_WAS_INITIATED                                            16971 non-null  object
 7   RECORD_STATUS_CODE                                            16971 non-null

In [16]:
nypd_data.describe()

,STOP_ID,YEAR2,ISSUING_OFFICER_COMMAND_CODE,SUPERVISING_OFFICER_COMMAND_CODE,OBSERVED_DURATION_MINUTES,STOP_DURATION_MINUTES,STOP_LOCATION_PRECINCT
count,16971.000000,16971.0,16971.000000,16971.000000,16971.000000,16971.000000,16971.000000
mean,8486.000000,2023.0,149.460609,154.172353,3.637322,8.231041,58.719109
std,4899.250045,0.0,233.567125,234.754082,179.708902,13.395769,30.395536
min,1.000000,2023.0,1.000000,1.000000,0.000000,0.000000,1.000000
25%,4243.500000,2023.0,44.000000,44.000000,1.000000,2.000000,41.000000
50%,8486.000000,2023.0,61.000000,66.000000,1.000000,5.000000,48.000000
75%,12728.500000,2023.0,109.000000,113.000000,2.000000,10.000000,78.000000
max,16971.000000,2023.0,879.000000,881.000000,23042.000000,581.000000,123.000000


In [19]:
nypd_data.isna().sum()

STOP_ID                            0
STOP_FRISK_DATE                    0
STOP_FRISK_TIME                    0
YEAR2                              0
MONTH2                             0
                                  ..
STOP_LOCATION_STREET_NAME         18
STOP_LOCATION_X                    0
STOP_LOCATION_Y                    0
STOP_LOCATION_PATROL_BORO_NAME     0
STOP_LOCATION_BORO_NAME            0
Length: 82, dtype: int64